In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # I use a HiDPI screen

In [10]:

import os
import numpy as np
import matplotlib.pyplot as plt
import file_loading
from scipy.interpolate import Rbf
from multiprocessing import Pool
from astropy.io import ascii
from forecast import build_interp_zack
from forecast import findlevel
from forecast import plot_cube

In [11]:
nres = 50
computing_PS_flag = True
computing_PC_flag = True

### 1. PS/PC, cut at ellmax=5000, for single redshift (ngal40)

In [21]:
%%time
output_cube_name = '1ngal40'
data_dir = '/home/zequnl/Projects/neutrino_mpk/'
table = ascii.read('parameters.table')

params, obsarr_PS, obsarr_PC, ell, kappa, invcov_PS, invcov_PC = \
                file_loading.get_data_arrays_across_redshifts( table,
                    data_dir = data_dir,
                    PS_directory = data_dir + 'powerspectrum_ngal40', 
                    PC_directory = data_dir + 'peakcounts_ngal40',
                    redshifts = ['10'])

# we overwrite the inverse covariance matrix with the new one
# NOTE: ngal40 requires a special string!
# invcov_PC = file_loading.compute_custom_PC_cov(  ['10'], 
#      fid_string='Om0.29997_As2.10000_mva0.00000_mvb0.00000_mvc0.00000_h0.70000_Ode0.69995_PC_S_z10_cov_ngal40.npy')

    
# set up grid

x_PS = np.linspace(0.1 - 0.04, 0.1 + 0.13, nres) # M_nu
y_PS = np.linspace(0.3 - 0.012, 0.3 + 0.01, nres) # omega_m
z_PS = np.linspace(0.8295 - 0.012, 0.8295 + 0.016, nres) # sigma_8
X_PS, Y_PS, Z_PS = np.meshgrid(x_PS, y_PS, z_PS, indexing='ij')
input_param_list_PS = list(zip(np.ravel(X_PS), np.ravel(Y_PS), np.ravel(Z_PS)))

x_PC = np.linspace(0.1 - 0.01, 0.1 + 0.01, nres) # M_nu
y_PC = np.linspace(0.3 - 0.003, 0.3 + 0.003, nres) # omega_m
z_PC = np.linspace(0.8295 - 0.005, 0.8295 + 0.005, nres) # sigma_8
X_PC, Y_PC, Z_PC = np.meshgrid(x_PC, y_PC, z_PC, indexing='ij')
input_param_list_PC = list(zip(np.ravel(X_PC), np.ravel(Y_PC), np.ravel(Z_PC)))

# ----- BOILER PLATE STUFF BEGINS HERE

# set up interpolator
interp_PS = build_interp_zack(obsarr_PS, params)
interp_PC = build_interp_zack(obsarr_PC, params)

def P_PS( parameter_input ):
    my_interp = interp_PS;
    my_invcov = invcov_PS
    my_fid_model = obsarr_PS[1]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

def P_PC( parameter_input ):
    my_interp = interp_PC
    my_invcov = invcov_PC
    my_fid_model = obsarr_PC[1]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

# if __name__ == '__main__':
#     if computing_PS_flag:
#         pool = Pool(16)
#         PS_cube = np.array(pool.map(P_PS, input_param_list_PS))
#         PS_cube = PS_cube.reshape(X_PS.shape)
#         pool.close()
#         np.save( 'cubes/' + output_cube_name + '_PS.npy', PS_cube )
#         np.save( 'cubes/' + output_cube_name + '_PS_xyz.npy', np.array([x_PS,y_PS,z_PS]) )
        
#     if computing_PC_flag:
#         pool = Pool(16)
#         PC_cube = np.array(pool.map(P_PC, input_param_list_PC))
#         PC_cube = PC_cube.reshape(X_PC.shape)
#         pool.close()
#         np.save( 'cubes/' + output_cube_name + '_PC.npy', PC_cube )
#         np.save( 'cubes/' + output_cube_name + '_PC_xyz.npy', np.array([x_PC,y_PC,z_PC]) )

('PS bins', [24])
('PC bins', [184])
('PS nr', 1000, 'nb', 24, 0)
('PC nr', 1000, 'nb', 184, 0)


ValueError: All arrays must be equal length.

In [30]:

from scipy import interpolate, stats
function='multiquadric'
smooth=0.0

print(params.shape)

# # create a list of Rbf for each independent mode
# spline_interps = [ interpolate.Rbf(params.T, model, 
#                    function=function, smooth=smooth) for model in obsarr_PS.T ]
interpolate.Rbf(*params.T, obsarr_PS.T[0]) 
#     # return a function that applies Rbf to the parameters given, for each mode
#     return lambda params: np.array([ii(*params) for ii in spline_interps])

SyntaxError: only named arguments may follow *expression (<ipython-input-30-f725deabdb24>, line 11)

In [29]:
params

array([[ 0.     ,  0.3    ,  0.8523 ],
       [ 0.1    ,  0.3    ,  0.8295 ],
       [ 0.06271,  0.3815 ,  1.0135 ],
       [ 0.06522,  0.2821 ,  0.7563 ],
       [ 0.06773,  0.4159 ,  0.9171 ],
       [ 0.07024,  0.2023 ,  0.6231 ],
       [ 0.07275,  0.3283 ,  0.9324 ],
       [ 0.07526,  0.3355 ,  0.7828 ],
       [ 0.07778,  0.2597 ,  0.8008 ],
       [ 0.0803 ,  0.2783 ,  0.8382 ],
       [ 0.08282,  0.2758 ,  0.7285 ],
       [ 0.08535,  0.3132 ,  0.8378 ],
       [ 0.08788,  0.2926 ,  0.7429 ],
       [ 0.09041,  0.2845 ,  0.8126 ],
       [ 0.09295,  0.3155 ,  0.9411 ],
       [ 0.0955 ,  0.359  ,  1.0231 ],
       [ 0.09805,  0.3006 ,  0.8059 ],
       [ 0.10061,  0.2796 ,  0.769  ],
       [ 0.10318,  0.3977 ,  1.0456 ],
       [ 0.10575,  0.3051 ,  0.8004 ],
       [ 0.10833,  0.2833 ,  0.8385 ],
       [ 0.11092,  0.3763 ,  0.9574 ],
       [ 0.11351,  0.1841 ,  0.4885 ],
       [ 0.11612,  0.3085 ,  0.7821 ],
       [ 0.11874,  0.3143 ,  0.8358 ],
       [ 0.12136,  0.3457